# HexaSudoku Puzzle Generator

Generates valid 16×16 Sudoku puzzles (4×4 boxes) using Z3 SAT solver.
Uses values 1-16, displayed as 1-9 and A-F (hexadecimal notation).

## Installations

In [ ]:
!pip install z3-solver

In [ ]:
from z3 import *
import numpy as np
import random
import time
import json
import os

## Constants

In [ ]:
SIZE = 16
BOX_SIZE = 4  # 4x4 boxes for 16x16 grid
NUM_PUZZLES = 100
MIN_CLUES = 55  # Minimum clues for unique solution (estimated)

## Helper: Value to Character Conversion

In [ ]:
def value_to_char(value):
    """Convert 1-16 to display character (1-9, A-F, or . for 0)."""
    if value == 0:
        return '.'
    if value <= 9:
        return str(value)
    return chr(ord('A') + value - 10)  # 10→A, 11→B, ..., 16→G? No, 15→F

def print_grid(grid):
    """Print a 16x16 grid with hex notation."""
    for i, row in enumerate(grid):
        if i > 0 and i % BOX_SIZE == 0:
            print('-' * 37)
        line = ''
        for j, val in enumerate(row):
            if j > 0 and j % BOX_SIZE == 0:
                line += '| '
            line += value_to_char(val) + ' '
        print(line)

## Sudoku Solver with Z3

In [ ]:
def solve_sudoku(given_cells):
    """
    Solve a 16x16 Sudoku puzzle using Z3.
    
    Args:
        given_cells: dict of {(row, col): value} for pre-filled cells
    
    Returns:
        Solution grid or None if unsolvable
    """
    # Create variables
    X = [[Int(f"x_{i}_{j}") for j in range(SIZE)] for i in range(SIZE)]
    
    s = Solver()
    
    # Cell range constraints: 1 <= X[i][j] <= 16
    for i in range(SIZE):
        for j in range(SIZE):
            s.add(And(X[i][j] >= 1, X[i][j] <= SIZE))
    
    # Given cell constraints
    for (i, j), val in given_cells.items():
        s.add(X[i][j] == val)
    
    # Row constraints: each row has distinct values
    for i in range(SIZE):
        s.add(Distinct([X[i][j] for j in range(SIZE)]))
    
    # Column constraints: each column has distinct values
    for j in range(SIZE):
        s.add(Distinct([X[i][j] for i in range(SIZE)]))
    
    # Box constraints: each 4x4 box has distinct values
    for box_row in range(BOX_SIZE):
        for box_col in range(BOX_SIZE):
            cells = []
            for i in range(BOX_SIZE):
                for j in range(BOX_SIZE):
                    cells.append(X[box_row * BOX_SIZE + i][box_col * BOX_SIZE + j])
            s.add(Distinct(cells))
    
    if s.check() == sat:
        m = s.model()
        solution = [[m.evaluate(X[i][j]).as_long() for j in range(SIZE)] for i in range(SIZE)]
        return solution
    else:
        return None

In [ ]:
def has_unique_solution(given_cells):
    """
    Check if a 16x16 Sudoku puzzle has exactly one solution.
    """
    X = [[Int(f"x_{i}_{j}") for j in range(SIZE)] for i in range(SIZE)]
    s = Solver()
    
    # Cell range constraints
    for i in range(SIZE):
        for j in range(SIZE):
            s.add(And(X[i][j] >= 1, X[i][j] <= SIZE))
    
    # Given cell constraints
    for (i, j), val in given_cells.items():
        s.add(X[i][j] == val)
    
    # Row constraints
    for i in range(SIZE):
        s.add(Distinct([X[i][j] for j in range(SIZE)]))
    
    # Column constraints
    for j in range(SIZE):
        s.add(Distinct([X[i][j] for i in range(SIZE)]))
    
    # Box constraints
    for box_row in range(BOX_SIZE):
        for box_col in range(BOX_SIZE):
            cells = []
            for i in range(BOX_SIZE):
                for j in range(BOX_SIZE):
                    cells.append(X[box_row * BOX_SIZE + i][box_col * BOX_SIZE + j])
            s.add(Distinct(cells))
    
    # Find first solution
    if s.check() != sat:
        return False
    
    first_solution = s.model()
    
    # Block first solution and check for another
    block = []
    for i in range(SIZE):
        for j in range(SIZE):
            block.append(X[i][j] != first_solution.evaluate(X[i][j]))
    s.add(Or(block))
    
    # If no second solution exists, puzzle is unique
    return s.check() == unsat

## Generate Complete 16x16 Grid

In [ ]:
def generate_complete_grid():
    """
    Generate a complete, valid 16x16 Sudoku grid using Z3 with randomization.
    """
    X = [[Int(f"x_{i}_{j}") for j in range(SIZE)] for i in range(SIZE)]
    s = Solver()
    
    # Cell range constraints
    for i in range(SIZE):
        for j in range(SIZE):
            s.add(And(X[i][j] >= 1, X[i][j] <= SIZE))
    
    # Row constraints
    for i in range(SIZE):
        s.add(Distinct([X[i][j] for j in range(SIZE)]))
    
    # Column constraints
    for j in range(SIZE):
        s.add(Distinct([X[i][j] for i in range(SIZE)]))
    
    # Box constraints
    for box_row in range(BOX_SIZE):
        for box_col in range(BOX_SIZE):
            cells = []
            for i in range(BOX_SIZE):
                for j in range(BOX_SIZE):
                    cells.append(X[box_row * BOX_SIZE + i][box_col * BOX_SIZE + j])
            s.add(Distinct(cells))
    
    # Add some random constraints to get varied solutions
    random_cells = random.sample([(i, j) for i in range(SIZE) for j in range(SIZE)], 5)
    for (i, j) in random_cells:
        val = random.randint(1, SIZE)
        s.add(X[i][j] == val)
    
    if s.check() == sat:
        m = s.model()
        return [[m.evaluate(X[i][j]).as_long() for j in range(SIZE)] for i in range(SIZE)]
    else:
        # If random constraints conflict, try again without them
        return generate_complete_grid()

## Generate Puzzle by Removing Cells

In [ ]:
def generate_puzzle(min_clues=MIN_CLUES, max_clues=128):
    """
    Generate a 16x16 Sudoku puzzle by starting with a complete grid and removing cells
    while maintaining unique solution.
    
    Args:
        min_clues: Minimum number of given cells (default: 55)
        max_clues: Maximum number of given cells (default: 128 = half of 256)
    
    Returns:
        (puzzle, solution) tuple where puzzle has 0s for empty cells
    """
    # Generate complete solution
    solution = generate_complete_grid()
    
    # Start with all cells as given
    puzzle = [row[:] for row in solution]
    given_cells = {(i, j): solution[i][j] for i in range(SIZE) for j in range(SIZE)}
    
    # Randomly order cells for removal
    cells_to_try = [(i, j) for i in range(SIZE) for j in range(SIZE)]
    random.shuffle(cells_to_try)
    
    for (i, j) in cells_to_try:
        if len(given_cells) <= min_clues:
            break
        
        # Try removing this cell
        val = given_cells.pop((i, j))
        
        # Check if puzzle still has unique solution
        if has_unique_solution(given_cells):
            puzzle[i][j] = 0  # Mark as empty
        else:
            # Put it back
            given_cells[(i, j)] = val
    
    return puzzle, solution

## Generate Puzzle Set

In [ ]:
def generate_puzzle_set(num_puzzles):
    """
    Generate a set of 16x16 Sudoku puzzles.
    
    Args:
        num_puzzles: Number of puzzles to generate
    
    Returns:
        List of {"puzzle": [...], "solution": [...]} dicts
    """
    puzzles = []
    
    for i in range(num_puzzles):
        start = time.time()
        puzzle, solution = generate_puzzle()
        elapsed = time.time() - start
        
        clues = sum(1 for row in puzzle for cell in row if cell != 0)
        puzzles.append({"puzzle": puzzle, "solution": solution})
        
        print(f"Generated puzzle {i+1}/{num_puzzles} ({clues} clues, {elapsed:.1f}s)")
    
    return puzzles

## Demo: Generate Single Puzzle

In [ ]:
# Generate a single 16x16 puzzle
print("Generating a single 16x16 HexaSudoku puzzle...")
start = time.time()
puzzle, solution = generate_puzzle()
elapsed = time.time() - start

clues = sum(1 for row in puzzle for cell in row if cell != 0)
print(f"\nGenerated puzzle with {clues} clues in {elapsed:.1f}s")

print("\n16x16 Puzzle:")
print_grid(puzzle)

print("\nSolution:")
print_grid(solution)

## Generate Full Dataset (100 puzzles)

In [ ]:
print(f"Generating {NUM_PUZZLES} 16x16 HexaSudoku puzzles...")
print("This may take a while (16x16 puzzles are computationally intensive)")
print()

start_total = time.time()
puzzles = generate_puzzle_set(NUM_PUZZLES)
total_time = time.time() - start_total

print(f"\nTotal time: {total_time:.1f}s ({total_time/NUM_PUZZLES:.1f}s per puzzle)")

## Analyze Dataset

In [ ]:
# Analyze clue distribution
clue_counts = [sum(1 for row in p['puzzle'] for cell in row if cell != 0) for p in puzzles]

print(f"Clue distribution for {len(puzzles)} puzzles:")
print(f"  Min: {min(clue_counts)}")
print(f"  Max: {max(clue_counts)}")
print(f"  Mean: {np.mean(clue_counts):.1f}")
print(f"  Std: {np.std(clue_counts):.1f}")

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.hist(clue_counts, bins=20, edgecolor='black')
plt.title('Distribution of Clue Counts in 16x16 HexaSudoku Puzzles')
plt.xlabel('Number of Clues')
plt.ylabel('Frequency')
plt.axvline(np.mean(clue_counts), color='red', linestyle='--', label=f'Mean: {np.mean(clue_counts):.1f}')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Save Dataset

In [ ]:
# Prepare data for JSON serialization
output_data = {
    "16": puzzles
}

# Save to file
output_path = "./puzzles/puzzles_dict.json"
os.makedirs(os.path.dirname(output_path), exist_ok=True)

with open(output_path, "w") as f:
    json.dump(output_data, f, indent=2)

print(f"Saved {len(puzzles)} puzzles to {output_path}")

## Verify Dataset

In [ ]:
# Load and verify
with open(output_path, "r") as f:
    loaded = json.load(f)

print("Dataset summary:")
for size, puzzle_list in loaded.items():
    print(f"  {size}×{size}: {len(puzzle_list)} puzzles")
    
    # Count average clues
    total_clues = 0
    for p in puzzle_list:
        clues = sum(1 for row in p['puzzle'] for cell in row if cell != 0)
        total_clues += clues
    avg_clues = total_clues / len(puzzle_list)
    print(f"         Average clues: {avg_clues:.1f}")